In [1]:
import os

In [2]:
%pwd

'c:\\Users\\pcx\\Desktop\\Projects\\muru1\\Muru-Car\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\pcx\\Desktop\\Projects\\muru1\\Muru-Car'

In [5]:
import pandas as pd

In [6]:
data = pd.read_csv("artifacts/data_cleaning/new_df.csv")
data.head()

,Engine rpm,Lub oil pressure,Fuel pressure,Coolant pressure,lub oil temp,Coolant temp,Engine Condition
0,890,2.460128,10.340059,2.614929,85.015546,84.697231,1
1,1213,1.738052,6.957437,1.822885,76.813195,93.325762,1
2,719,2.701037,4.595192,5.117364,77.057828,86.119871,0
3,892,4.522354,9.248958,2.698915,75.074354,82.423507,0
4,851,5.945380,7.395428,2.201839,84.180636,79.454742,0


In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19535 entries, 0 to 19534
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Engine rpm        19535 non-null  int64  
 1   Lub oil pressure  19535 non-null  float64
 2   Fuel pressure     19535 non-null  float64
 3   Coolant pressure  19535 non-null  float64
 4   lub oil temp      19535 non-null  float64
 5   Coolant temp      19535 non-null  float64
 6   Engine Condition  19535 non-null  int64  
dtypes: float64(5), int64(2)
memory usage: 1.0 MB


In [8]:
data.isnull().sum()

Engine rpm          0
Lub oil pressure    0
Fuel pressure       0
Coolant pressure    0
lub oil temp        0
Coolant temp        0
Engine Condition    0
dtype: int64

In [9]:
data.shape

(19535, 7)

In [10]:
data.keys()

Index(['Engine rpm', 'Lub oil pressure', 'Fuel pressure', 'Coolant pressure',
       'lub oil temp', 'Coolant temp', 'Engine Condition'],
      dtype='object')

In [11]:
data.columns

Index(['Engine rpm', 'Lub oil pressure', 'Fuel pressure', 'Coolant pressure',
       'lub oil temp', 'Coolant temp', 'Engine Condition'],
      dtype='object')

In [12]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataValidationConfig:
    root_dir: Path
    STATUS_FILE: str
    unzip_data_dir: Path
    all_schema: dict

In [13]:
from mlProject.constants import *
from mlProject.utils.common import read_yaml, create_directories

In [14]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_validation_config(self) -> DataValidationConfig:
        config = self.config.data_validation
        schema = self.schema.COLUMNS

        create_directories([config.root_dir])

        data_validation_config = DataValidationConfig(
            root_dir=config.root_dir,
            STATUS_FILE=config.STATUS_FILE,
            unzip_data_dir = config.unzip_data_dir,
            all_schema=schema,
        )

        return data_validation_config

In [15]:
import os
from mlProject import logger

In [16]:
import pandas as pd

class DataValidation:
    def __init__(self, config: DataValidationConfig):
        self.config = config

    def validate_all_columns(self) -> bool:
        try:
            validation_status = True  # Assume validation passes initially

            data = pd.read_csv(self.config.unzip_data_dir)
            all_cols = list(data.columns)
            all_schema = self.config.all_schema.keys()

            print("Columns in dataset:", all_cols)
            print("Columns in schema:", all_schema)

            for col in all_schema:
                if col not in all_cols:
                    validation_status = False
                    print(f"Column '{col}' not found in dataset.")
                    break

            # Write the final validation status after the loop
            with open(self.config.STATUS_FILE, 'w') as f:
                f.write(f"Validation status: {validation_status}")

            return validation_status

        except Exception as e:
            print("An exception occurred:", e)
            raise e


In [17]:
try:
    config = ConfigurationManager()
    data_validation_config = config.get_data_validation_config()
    data_validation = DataValidation(config=data_validation_config)
    data_validation.validate_all_columns()
except Exception as e:
    raise e

[2023-08-18 16:40:45,887: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-08-18 16:40:45,895: INFO: common: yaml file: params.yaml loaded successfully]
[2023-08-18 16:40:45,901: INFO: common: yaml file: schema.yaml loaded successfully]
[2023-08-18 16:40:45,911: INFO: common: created directory at: artifacts]
[2023-08-18 16:40:45,911: INFO: common: created directory at: artifacts/data_validation]
Columns in dataset: ['Engine rpm', 'Lub oil pressure', 'Fuel pressure', 'Coolant pressure', 'lub oil temp', 'Coolant temp', 'Engine Condition']
Columns in schema: dict_keys(['Engine rpm', 'Lub oil pressure', 'Fuel pressure', 'Coolant pressure', 'lub oil temp', 'Coolant temp'])
